In [2]:
import datasets
import os
import pandas as pd

import transformers
import torch
import tensorflow as tf;
import re
from pathlib import Path
import subprocess
import json
from tqdm.notebook import tqdm


REPOS_DIR = os.getcwd()+'/data/repos'

gvm_root = os.environ['GVM_ROOT']
os.environ['PATH'] = f"{gvm_root}/bin:{gvm_root}/pkgsets/go1.24.2/global/bin:{gvm_root}/gos/go1.24.2/bin:{gvm_root}/pkgsets/go1.24.2/global/overlay/bin:{os.environ['PATH']}"
goimports_binary = f"{gvm_root}/pkgsets/go1.24.2/global/bin/goimports"

2025-05-01 20:29:37.838403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746120577.926261 3064202 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746120577.951557 3064202 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746120578.141325 3064202 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746120578.141350 3064202 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746120578.141351 3064202 computation_placer.cc:177] computation placer alr

Results

100 eval
{'eval_loss': 0.008647013455629349, 'eval_runtime': 1728.1625, 'eval_samples_per_second': 0.058, 'eval_steps_per_second': 0.029}

train 100
{'train_runtime': 6765.2233, 'train_samples_per_second': 0.044, 'train_steps_per_second': 0.044, 'train_loss': 0.0048288830121358236, 'num_tokens': 837861.0, 'completions/mean_length': 564.225, 'completions/min_length': 430.6566666666667, 'completions/max_length': 697.7933333333333, 'completions/clipped_ratio': 0.045, 'completions/mean_terminated_length': 531.115, 'completions/min_terminated_length': 430.6566666666667, 'completions/max_terminated_length': 631.5733333333334, 'rewards/reward_func/mean': 0.07412733356157938, 'rewards/reward_func/std': 0.051573539904687396, 'reward': 0.07412733356157938, 'reward_std': 0.051573539904687396, 'kl': 0.0031781005859375, 'clip_ratio/low_mean': 0.0, 'clip_ratio/low_min': 0.0, 'clip_ratio/high_mean': 0.0, 'clip_ratio/high_max': 0.0, 'clip_ratio/region_mean': 0.0, 'epoch': 3.0}

500 eval
{'eval_loss': 0.0038519210647791624, 'eval_runtime': 8757.9875, 'eval_samples_per_second': 0.057, 'eval_steps_per_second': 0.029}

trained by 100 model

500 eval
{'eval_loss': 0.005923781078308821, 'eval_runtime': 8298.6012, 'eval_samples_per_second': 0.06, 'eval_steps_per_second': 0.03}

learn 200
{'loss': 0.0034, 'grad_norm': 1.8515625, 'learning_rate': 1.6833333333333332e-07, 'num_tokens': 1410261.0, 'completions/mean_length': 514.533, 'completions/min_length': 389.576, 'completions/max_length': 639.49, 'completions/clipped_ratio': 0.04, 'completions/mean_terminated_length': 483.142, 'completions/min_terminated_length': 384.776, 'completions/max_terminated_length': 581.508, 'rewards/reward_func/mean': 0.08201166647672653, 'rewards/reward_func/std': 0.07028801679419121, 'reward': 0.08201166647672653, 'reward_std': 0.07028801679430763, 'kl': 0.0033938522338867187, 'clip_ratio/low_mean': 0.0, 'clip_ratio/low_min': 0.0, 'clip_ratio/high_mean': 0.0, 'clip_ratio/high_max': 0.0, 'clip_ratio/region_mean': 0.0, 'epoch': 2.5}

500 train
{'loss': 0.0001, 'grad_norm': 0.0016553926980122924, 'learning_rate': 8e-08, 'num_tokens': 2844345.0, 'completions/mean_length': 305.0, 'completions/min_length': 203.0, 'completions/max_length': 355.0, 'completions/clipped_ratio': 0.0, 'completions/mean_terminated_length': 305.0, 'completions/min_terminated_length': 203.0, 'completions/max_terminated_length': 355.0, 'rewards/reward_func/mean': 0.0, 'rewards/reward_func/std': 0.0, 'reward': 0.0, 'reward_std': 0.0, 'kl': 0.003600830852519721, 'clip_ratio/low_mean': 0.0, 'clip_ratio/low_min': 0.0, 'clip_ratio/high_mean': 0.0, 'clip_ratio/high_max': 0.0, 'clip_ratio/region_mean': 0.0, 'epoch': 2.0}
{'train_runtime': 38069.9954, 'train_samples_per_second': 0.026, 'train_steps_per_second': 0.007, 'train_loss': 0.004397845288476674, 'epoch': 2.0}

In [1]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
# !pip install transformers
# !pip install tensorflow[and-cuda]
# !pip install tf-keras
!pip install trl

  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.7.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.7.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.5.4.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_6

In [3]:
ds = datasets.Dataset.load_from_disk('data/test_candidates_ds')

print(len(ds))

ds[0]

3604


{'path': '/main.go',
 'repo_name': 'avelino/awesome-go',
 'revision_id': 'c3643eb9da5c673101f8fe15a6deb40bfc4a1c85',
 'star_events_count': 112752,
 'fork_events_count': 13739,
 'is_vendor': False,
 'is_generated': False,
 'length_bytes': 8755,
 'extension': 'go',
 'filename': 'main.go',
 'file_exists': True,
 'file_hash': '105132c75e5c98fa0379971e18dd1bb6544cabc7c5b18a81ae44585a4db25705',
 'project': '/',
 'relative_project_path': 'main.go',
 'relative_go_package': '',
 'funcs': 'main,buildStaticSite,dropCreateDir,mkdirAll,renderCategories,renderSitemap,extractCategories,extractCategory,rewriteLinksInIndex,renderIndex',
 'methods': '',
 'project_path': 'c3643eb9da5c673101f8fe15a6deb40bfc4a1c85/'}

In [4]:
import gc

# try:
#     del tokenizer
#     del model
# except Exception:
#     pass

#model.cpu()
#del tokenizer, model

gc.collect()
torch.cuda.empty_cache()

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [5]:
# model_name = 'deepseek-ai/deepseek-coder-6.7b-base'
# model_name = 'deepseek-ai/deepseek-coder-6.7b-instruct'
# model_name = 'deepseek-ai/deepseek-coder-1.3b-base'
model_name = 'deepseek-ai/deepseek-coder-1.3b-instruct'

# tokenizer = transformers.AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
# model = transformers.AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()

In [6]:
def generate_unit_test(file_content: str) -> str:
    system_message = """
You are an expert programmer. 
You should only return output test file containing working code.
The user is going to give you code and would like to have unit tests for the first file.
All the other files are just dependencies to give you context of all the possible test cases to produce.
Cover all possible inputs and their respective outputs using tests.
Each subtest must be wrapped into t.Run
"""

    code = f"{file_content}"


    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": code},
    ]
    # https://huggingface.co/docs/transformers/main/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.apply_chat_template
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt", truncation=True, max_length=16000).to(model.device)
    # tokenizer.eos_token_id is the id of <|EOT|> token
    model.generation_config.pad_token_id = tokenizer.pad_token_id
    outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50,
        # top_p=0.95,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,eos_token_id=tokenizer.eos_token_id)
    
    answer: str = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

    go_content_start = answer.find('```go')+5
    go_content_end = answer.find('```', go_content_start)
    test_file_content = answer[go_content_start:go_content_end]
    return test_file_content

In [130]:
for row in tqdm(ds):
    project_path = REPOS_DIR+'/'+row['project_path']
    for p in Path(project_path).glob("**/*_test.go"):
        p.unlink()

  0%|          | 0/3604 [00:00<?, ?it/s]

In [131]:
def check_result(row, test_file_content):
    project_path = REPOS_DIR+'/'+row['project_path']
    test_relative_project_path = row['relative_project_path'][:len(row['relative_project_path'])-3]+'_llm_test.go'
    test_file_path = project_path+test_relative_project_path
    cover_file_path = project_path+row['relative_project_path'][:len(row['relative_project_path'])-3]+'_llm_test_cover.out'

    for p in Path(project_path).glob("**/*_test.go"):
        p.unlink()

    with open(test_file_path, "w") as f:
        f.write(test_file_content)

    test_fns: list[str] = re.findall(r'func\W+(Test\w+)\(t\W+\*testing\.T\)\W*\{', test_file_content)

    p = subprocess.run([goimports_binary, "-w", test_file_path], cwd=project_path,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
    )
    if p.returncode != 0:
        raise Exception(f"goimports: code {p.returncode}, 'strderr' {p.stderr}")

    # print('# main_file_content:', main_file_content, '# test_file_content', test_file_content, '# test_fns', test_fns, sep="\n\n")

    p = subprocess.run(["go", "mod", "tidy"], cwd=project_path,
                        stdout=subprocess.PIPE, 
                        stderr=subprocess.PIPE,
    )
    if p.returncode != 0:
        raise Exception(f"go mod tidy: code {p.returncode}, 'strderr' {p.stderr}")
        # print('go mod tidy: code', p.returncode, 'strderr', p.stderr)

    test_stderr = ''
    p = subprocess.run(["go", "test", "./"+row['relative_go_package'], '-run', '/'.join(test_fns), '-json', '-coverprofile', cover_file_path], cwd=project_path,
                        stdout=subprocess.PIPE, 
                        stderr=subprocess.PIPE,
    )
    #print("debug go test", p.returncode, p.stdout.decode("utf-8"), p.stderr.decode("utf-8"), "", sep="\n")
    all_passed = True
    if p.returncode != 0:
        all_passed = False
        test_stderr = p.stderr
        #print('code', p.returncode, 'strderr', p.stderr.decode("utf-8"), 'stdout', p.stdout.decode("utf-8"))
    passed = 0
    failed = 0
    root_passed = 0
    root_failed = 0
    for line in p.stdout.decode('utf-8').split('\n'):
        if line == "":
            continue
        line_data = json.loads(line)
        if line_data.get('Action') == 'build-fail':
            raise Exception(f"go test build failed: stdout = {p.stdout}, stderr = {p.stderr}")
            {"Time":"2025-04-29T20:35:13.11374012+03:00","Action":"fail","Package":"k8s.io/kops/tests/e2e/pkg/kops","Test":"TestGetCluster","Elapsed":0}
        if 'Test' not in line_data:
            continue
        if 'Action' not in line_data:
            continue
        action = line_data['Action']
        if action == 'pass':
            if '/' in line_data['Test']:
                passed += 1
            else:
                root_passed += 1
        elif action == 'fail':
            if '/' in line_data['Test']:
                failed += 1
            else:
                root_failed += 1
        else:
            continue
            #print(json.loads(line))
            #data.append(json.loads(line))
        #print(f"cd {project_path}")
        #print(' '.join(["go", "test", "./"+row['relative_go_package'], '-run', '/'.join(test_fns)]))

    f = open(cover_file_path)
    coverout = f.read()
    f.close()

    f = open(cover_file_path, 'w')
    f.writelines([line+'\n' for line in coverout.split('\n') if row['relative_project_path'] in line or line.startswith('mode:')])
    f.close()

    p = subprocess.run(["go", "tool", "cover", "-func", cover_file_path], cwd=project_path,
                        stdout=subprocess.PIPE, 
                        stderr=subprocess.PIPE,
    )
    if p.returncode != 0:
        print('go tool cover: code', p.returncode, 'strderr', p.stderr)

    coverage = 0.0
    foundCoverage = False
    for line in p.stdout.decode('utf-8').split('\n'):
        if line.startswith('total:'):
            out = re.search(r"\(statements\)\W+(\d+\.?\d*)\%", line)
            coverage = float(out.group(1))
            foundCoverage = True
    if not foundCoverage:
        print('no total coverage, stdout:', p.stdout, 'stderr:', p.stderr)

    # clean
    os.remove(test_file_path)
    os.remove(cover_file_path)

    return {'root_passed': root_passed, 'root_failed': root_failed, "passed": passed, "failed": failed, "all_passed": all_passed, "coverage": coverage, 'test_stderr': test_stderr}

In [122]:
cache = {}

def process_file(row) -> dict:
    project_path = REPOS_DIR+'/'+row['project_path']
    main_file_path = project_path+row['relative_project_path']

    main_file_content = ''
    try:
        with open(main_file_path) as f:
            main_file_content = f.read()
    except Exception as e:
        error_str = str(e)
        error_type = 'read_main_file'
        return {'root_passed': 0, 'root_failed': 0, "passed": 0, "failed": 0, "all_passed": False, "coverage": 0, 'test_file_content': '', 'test_stderr': '', 'error_type': error_type, 'error': error_str}

    check_res = {}
    test_file_content = ''
    try:
        if row['file_hash'] in cache:
            test_file_content = cache[row['file_hash']]
        else:
            test_file_content = generate_unit_test(main_file_content)
            cache[row['file_hash']] = test_file_content
    except Exception as e:
        error_str = str(e)
        error_type = 'generate'
        return {'root_passed': 0, 'root_failed': 0, "passed": 0, "failed": 0, "all_passed": False, "coverage": 0, 'test_file_content': '', 'test_stderr': '', 'error_type': error_type, 'error': error_str}

    try:
        check_res = check_result(row, test_file_content)
    except Exception as e:
        error_str = str(e)
        error_type = 'other'
        if error_str.startswith('go mod tidy: '):
            error_type = 'deps'
        if error_str.startswith('goimports: '):
            error_type = 'goimports'
        if error_str.startswith('go test build failed: '):
            error_type = 'test_build_failed'
        #raise e
        return {'root_passed': 0, 'root_failed': 0, "passed": 0, "failed": 0, "all_passed": False, "coverage": 0, 'test_file_content': test_file_content, 'test_stderr': '', 'error_type': error_type, 'error': error_str}

    return {
        'root_passed': check_res.get('root_passed'),
        'root_failed': check_res.get('root_failed'),
        'passed': check_res.get('passed'),
        'failed': check_res.get('failed'),
        'all_passed': check_res.get('all_passed'),
        'coverage': check_res.get('coverage'),
        'test_file_content': test_file_content,
        'test_stderr': check_res.get('test_stderr'),
        'error_type': '',
        'error': '',
    }

In [132]:
row = {'path': '/pkg/markdown/convert.go',
 'repo_name': 'avelino/awesome-go',
 'revision_id': 'c3643eb9da5c673101f8fe15a6deb40bfc4a1c85',
 'star_events_count': 112752,
 'fork_events_count': 13739,
 'is_vendor': False,
 'is_generated': False,
 'length_bytes': 1342,
 'extension': 'go',
 'filename': 'convert.go',
 'file_exists': True,
 'file_hash': 'f0035e6fd1be46605da254d27a285c1ec41f081895fc73376b18219132a5f04d',
 'project': '/',
 'relative_project_path': 'pkg/markdown/convert.go',
 'relative_go_package': 'pkg/markdown/',
 'project_path': 'c3643eb9da5c673101f8fe15a6deb40bfc4a1c85/'}

row = ds.filter(lambda row: row['repo_name'] == 'avelino/awesome-go' and row['path'] == '/pkg/slug/generator.go')[0]
print(row)
#row = ds.filter(lambda row: row['repo_name'] == 'flipped-aurora/gin-vue-admin' and row['path'] == '/server/utils/ast/ast_auto_enter.go')[0]
#print(row)
# row = ds.filter(lambda row: row['repo_name'] == 'flipped-aurora/gin-vue-admin' and row['path'] == '/server/model/system/sys_dictionary_detail.go')[0]
# print(row)
row = ds.filter(lambda row: row['repo_name'] == 'dapr/dapr' and row['path'] == '/pkg/actors/actor_lock.go')[0]
print(row)

res = process_file(row)

print(res)
input_file = f"{REPOS_DIR}/{row['project_path']}{row['relative_project_path']}"
print(f"## input {row['repo_name']} {row['project_path']} {row['relative_go_package']} {input_file}")
with open(input_file) as f:
    print(f.read())
print('## output')
print(res['test_file_content'])
    

{'path': '/pkg/slug/generator.go', 'repo_name': 'avelino/awesome-go', 'revision_id': 'c3643eb9da5c673101f8fe15a6deb40bfc4a1c85', 'star_events_count': 112752, 'fork_events_count': 13739, 'is_vendor': False, 'is_generated': False, 'length_bytes': 326, 'extension': 'go', 'filename': 'generator.go', 'file_exists': True, 'file_hash': 'c7213fc665136d588378b02246f680ee3406572b8bbd20e2050aa8131cd9fdd5', 'project': '/', 'relative_project_path': 'pkg/slug/generator.go', 'relative_go_package': 'pkg/slug/', 'funcs': 'Generate', 'methods': '', 'project_path': 'c3643eb9da5c673101f8fe15a6deb40bfc4a1c85/'}
{'path': '/pkg/actors/actor_lock.go', 'repo_name': 'dapr/dapr', 'revision_id': 'f1950be0c4427da57024253572df171ee7bd2b68', 'star_events_count': 23122, 'fork_events_count': 2011, 'is_vendor': False, 'is_generated': False, 'length_bytes': 1816, 'extension': 'go', 'filename': 'actor_lock.go', 'file_exists': True, 'file_hash': '8df30f3932ce4b68e7c5760fd251bc4448b07dc70b3d278b82888e84bf09e319', 'project'

In [ ]:
res_ds = ds.shuffle().take(50).map(process_file, num_proc=1)

#res_ds.save_to_disk('data/res_ds')

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
res_ds.save_to_disk('data/res_ds')

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [87]:
#print(res_ds.to_pandas().query('error != ""')[['path', 'revision_id', 'repo_name', 'relative_go_package', 'error']])

for _, row in res_ds.to_pandas().query('error != ""').iterrows():
    print(row['repo_name'], f"{REPOS_DIR}/{row['project_path']}", row['relative_go_package'], row['path'], row['error'], row['test_file_content'], sep="\n")

dapr/dapr
/home/djvue/dev/vkr/data/repos/f1950be0c4427da57024253572df171ee7bd2b68/
pkg/actors/
/pkg/actors/actor_lock.go
go mod tidy: code 1, 'strderr' b'go: finding module for package github.com/dapr/dapr/pkg/pubsub\ngo: finding module for package github.com/dapr/dapr/pkg/utils\ngo: finding module for package github.com/dapr/dapr/pkg/pubsub/mocks\ngo: github.com/dapr/dapr/pkg/runtime/processor/pubsub tested by\n\tgithub.com/dapr/dapr/pkg/runtime/processor/pubsub.test imports\n\tgithub.com/dapr/dapr/pkg/pubsub: no matching versions for query "latest"\ngo: github.com/dapr/dapr/pkg/runtime/processor/pubsub tested by\n\tgithub.com/dapr/dapr/pkg/runtime/processor/pubsub.test imports\n\tgithub.com/dapr/dapr/pkg/pubsub/mocks: no matching versions for query "latest"\ngo: github.com/dapr/dapr/pkg/sentry/server/validator/internal tested by\n\tgithub.com/dapr/dapr/pkg/sentry/server/validator/internal.test imports\n\tgithub.com/dapr/dapr/pkg/utils: no matching versions for query "latest"\n'

pack

In [89]:
import numpy as np

# res_ds = datasets.load_from_disk('data/res_ds')

# print(res_ds.to_pandas()['passed'].value_counts())

# print(res_ds.to_pandas()['failed'].value_counts())

# print(res_ds.to_pandas()['all_passed'].value_counts())

# print(np.average(res_ds.to_pandas()['coverage']))

# print(len(res_ds.to_pandas().query('error != ""')))

total_files = len(res_ds)
success_files = len(res_ds.to_pandas().query('error == ""'))
error_files = len(res_ds.to_pandas().query('error != ""'))
all_passed_files = len(res_ds.to_pandas().query('all_passed'))
root_passed_tests = res_ds.to_pandas()['root_passed'].sum()
root_failed_tests = res_ds.to_pandas()['root_failed'].sum()
total_root_tests = root_passed_tests+root_failed_tests
passed_tests = res_ds.to_pandas()['passed'].sum()
failed_tests = res_ds.to_pandas()['failed'].sum()
total_tests = passed_tests+failed_tests
average_coverage = res_ds.to_pandas().query('error == ""')['coverage'].mean()

pd.DataFrame(data=[
    ['total_files', total_files, 1],
    ['success_files', success_files, success_files/total_files],
    ['error_files', error_files, error_files/total_files],
    ['all_passed_files', all_passed_files, all_passed_files/total_files],
    ['total_root_tests', total_root_tests, 1],
    ['root_passed_tests', root_passed_tests, root_passed_tests/total_root_tests],
    ['root_failed_tests', root_failed_tests, root_failed_tests/total_root_tests],
    ['total_tests', total_tests, 1],
    ['passed_tests', passed_tests, passed_tests/total_tests],
    ['failed_tests', failed_tests, failed_tests/total_tests],
    ['average_coverage', average_coverage, average_coverage],
], columns=['name', 'count', 'part']).set_index('name')

,count,part
name,,
total_files,100.000000,1.000000
success_files,7.000000,0.070000
error_files,93.000000,0.930000
all_passed_files,6.000000,0.060000
total_root_tests,9.000000,1.000000
root_passed_tests,8.000000,0.888889
root_failed_tests,1.000000,0.111111
total_tests,7.000000,1.000000
passed_tests,7.000000,1.000000


In [72]:
print(len(res_ds.to_pandas().query('error == "" and root_passed == 0 and root_failed == 0 and passed == 0 and failed == 0')))
for _, row in res_ds.to_pandas().query('error == "" and root_passed == 0 and root_failed == 0 and passed == 0 and failed == 0').head(10).iterrows():
    input_file = f"{REPOS_DIR}/{row['project_path']}{row['relative_project_path']}"
    print(f"## input {row['repo_name']} {row['project_path']} {input_file}")
    with open(input_file) as f:
        print(f.read())
    print('## output')
    print(row['test_file_content'])

1
## input etcd-io/etcd 2888a95d5d4f20e7e852214e888d79ead642300b/etcdctl/ /home/djvue/dev/vkr/data/repos/2888a95d5d4f20e7e852214e888d79ead642300b/etcdctl/ctlv3/command/get_command.go
// Copyright 2015 The etcd Authors
//
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
//     http://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

package command

import (
	"fmt"
	"strings"

	"github.com/spf13/cobra"

	clientv3 "go.etcd.io/etcd/client/v3"
	"go.etcd.io/etcd/pkg/v3/cobrautl"
)

var (
	getConsistency string
	getLimit       int64
	getSortOrder   string
	

In [76]:
#system_message = "The user is going to give you code and would like to have tests written in the Python unittest module."
# see https://developers.cloudflare.com/workers-ai/guides/tutorials/explore-code-generation-using-deepseek-coder-models/
# see docs deepseek-coder v2 https://github.com/deepseek-ai/DeepSeek-Coder-V2
# see https://github.com/deepseek-ai/DeepSeek-Coder
# see https://github.com/raiharsh88/tgen/blob/main/llama_server/generator/llama.py

# You are an expert programmer. 
# The user is going to give you code and would like to have tests
# Based on the following instructions and given code files, write unit tests for the first file;
# all the other files are just dependencies to give you context of all the possible test cases to produce.

# Instructions:
# 1. Each file wrapped between [File] and [/File] tags.
# 2. You should only return output test file.
# 3. Cover all possible inputs and their respective outputs using tests.
system_message = """
You are an expert programmer. 
You should only return output test file containing working code.
The user is going to give you code and would like to have unit tests for the first file.
All the other files are just dependencies to give you context of all the possible test cases to produce.
Cover all possible inputs and their respective outputs using tests.
"""

code = """
```python
#user.py
from password import Password

class User:

    def __init__(self, first_name, last_name=None, password: Password=None):
        self.first_name = first_name
        self.last_name = last_name
        self.password = password
        if last_name is None:
            self.last_name = "Mc" + self.first_name

    def full_name(self):
        return self.first_name + " " + self.last_name + " " + self.password.get_password()
```

```python
#password.py
class Password:

    def __init__(self, password):
        self._password = password

    def get_password():
        return self._password.lower()
```
"""

messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": code},
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
model.generation_config.pad_token_id = tokenizer.pad_token_id
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50,
    # top_p=0.95,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

Here is a set of unit tests for the `user.py` file.

```python
import unittest
from user import User
from password import Password

class TestUser(unittest.TestCase):

    def setUp(self):
        self.password = Password("TestPassword123")
        self.user = User("John", "Doe", self.password)

    def test_full_name(self):
        self.assertEqual(self.user.full_name(), "John Doe TestPassword123")

    def test_full_name_with_no_last_name(self):
        user = User("Jane")
        self.assertEqual(user.full_name(), "Jane McJane")

    def test_full_name_with_no_password(self):
        user = User("Jane", None)
        self.assertEqual(user.full_name(), "Jane McJane")

if __name__ == '__main__':
    unittest.main()
```

In this test suite, we are testing the `full_name` method of the `User` class. We are also testing the `full_name` method of the `User` class with no last name and no password.

The `setUp` method is used to create an instance of the `User` class and an instance of the

In [35]:
input_text = "#write a quick sort algorithm"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=1024)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


#write a quick sort algorithm in python


def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        less_than_pivot = [x for x in arr[1:] if x <= pivot]
        greater_than_pivot = [x for x in arr[1:] if x > pivot]
        return quick_sort(less_than_pivot) + [pivot] + quick_sort(greater_than_pivot)

print(quick_sort([3,6,8,10,1,2,1]))
#output: [1, 1, 2, 3, 6, 8, 10]

#This algorithm works by selecting a 'pivot' element from the array and partitioning the other elements into two sub-arrays, according to whether they are less than or greater than the pivot. The sub-arrays are then recursively sorted.

#The time complexity of the Quick Sort algorithm is O(n log n) in all 3 cases (worst, average and best) when the pivot is the smallest or the largest element.
#The space complexity is O(log n) in the best case and O(n) in the worst case.
#The worst case happens when the input array is already sorted.
#The best case happens when the pivot is the

In [39]:
input_text = """#utils.py
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

def load_data():
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target

    # Standardize the data
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Convert numpy data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.int64)
    y_test = torch.tensor(y_test, dtype=torch.int64)
    
    return X_train, X_test, y_train, y_test

def evaluate_predictions(y_test, y_pred):
    return accuracy_score(y_test, y_pred)
#model.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class IrisClassifier(nn.Module):
    def __init__(self):
        super(IrisClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 3)
        )

    def forward(self, x):
        return self.fc(x)

    def train_model(self, X_train, y_train, epochs, lr, batch_size):
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)
        
        # Create DataLoader for batches
        dataset = TensorDataset(X_train, y_train)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(epochs):
            for batch_X, batch_y in dataloader:
                optimizer.zero_grad()
                outputs = self(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

    def predict(self, X_test):
        with torch.no_grad():
            outputs = self(X_test)
            _, predicted = outputs.max(1)
        return predicted.numpy()
#main.py
from utils import load_data, evaluate_predictions
from model import IrisClassifier as Classifier

def main():
    # Model training and evaluation
"""
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=140)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


<｜begin▁of▁sentence｜>#utils.py
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

def load_data():
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target

    # Standardize the data
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Convert numpy data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.int64)
    y_test = torch.tensor(y_test, dtype=torch.int64)

    return X_train, X_test, y_train, y_test

def evaluate_predictions(y_test, y_pred):
    return accuracy_score(y_test, y_pred)
#model.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, 